In [4]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [12]:
"""
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if args.dry_run:
                break
"""

def train(model, device, train_loader, optimizer, epoch, log_interval, dry_run):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if dry_run:
                break

In [13]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [15]:
def main():
    # Training settings
    """
    parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
    parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
    parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                        help='input batch size for testing (default: 1000)')
    parser.add_argument('--epochs', type=int, default=14, metavar='N',
                        help='number of epochs to train (default: 14)')
    parser.add_argument('--lr', type=float, default=1.0, metavar='LR',
                        help='learning rate (default: 1.0)')
    parser.add_argument('--gamma', type=float, default=0.7, metavar='M',
                        help='Learning rate step gamma (default: 0.7)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--dry-run', action='store_true', default=False,
                        help='quickly check a single pass')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                        help='how many batches to wait before logging training status')
    parser.add_argument('--save-model', action='store_true', default=False,
                        help='For Saving the current Model')
    args = parser.parse_args()
    """
    # Parameter
    batch_size = 64
    test_batch_size = 1000
    epochs = 14
    lr = 1.0
    gamma = 0.7
    no_cuda = False
    dry_run = False
    seed = 1
    log_interval = 10
    save_model = False
    
    #use_cuda = not args.no_cuda and torch.cuda.is_available()
    use_cuda = not no_cuda and torch.cuda.is_available()

    #torch.manual_seed(args.seed)
    torch.manual_seed(seed)

    device = torch.device("cuda" if use_cuda else "cpu")

    #train_kwargs = {'batch_size': args.batch_size}
    #test_kwargs = {'batch_size': args.test_batch_size}
    train_kwargs = {'batch_size': batch_size}
    test_kwargs = {'batch_size': test_batch_size}
    
    if use_cuda:
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)

    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    dataset1 = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
    dataset2 = datasets.MNIST('../data', train=False,
                       transform=transform)
    train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

    model = Net().to(device)
    #optimizer = optim.Adadelta(model.parameters(), lr=args.lr)
    optimizer = optim.Adadelta(model.parameters(), lr=lr)

    #scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
    scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
    #for epoch in range(1, args.epochs + 1):
    for epoch in range(1, epochs + 1):
        #train(args, model, device, train_loader, optimizer, epoch)
        train(model, device, train_loader, optimizer, epoch, log_interval, dry_run)
        test(model, device, test_loader)
        scheduler.step()

    #if args.save_model:
    if save_model:
        torch.save(model.state_dict(), "mnist_cnn.pt")


if __name__ == '__main__':
    main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.318393
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.437988
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.882651
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.656152
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.581844
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.359730
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.273470
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.267700
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.354189
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.295077
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.296558
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.243722
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.289018
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.194202
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.311383
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.136868
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.209718
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.340396
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.299360
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.115921
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.015145
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.022694
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.114431
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.013198
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.010188
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.009995
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.011013
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.029652
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.091575
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.008427
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.066648
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.019360
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.009560
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.031330
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.189325
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.059104
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.063424
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.158514
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.009588
Train Epoch: 2 [58240/60000 (97%)]	Loss: 0.156753


Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.157587
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.001910
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.111325
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.010217
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.007644
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.016806
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.263808
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.160336
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.011831
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.065892
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.069577
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.047528
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.007894
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.068859
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.197285
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.018074
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.029021
Train Epoch: 4 [41600/60000 (69%)]	Loss: 0.013637
Train Epoch: 4 [42240/60000 (70%)]	Loss: 0.028703
Train Epoch: 4 [42880/60000 (71%)]	Loss: 0.003079


Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.003112
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.013594
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.020109
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.003278
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.024648
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.076623
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.054725
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.008536
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.003071
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.030952
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.004091
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.001591
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.015836
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.037510
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.001185
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.012384
Train Epoch: 6 [25600/60000 (43%)]	Loss: 0.026172
Train Epoch: 6 [26240/60000 (44%)]	Loss: 0.005874
Train Epoch: 6 [26880/60000 (45%)]	Loss: 0.010170
Train Epoch: 6 [27520/60000 (46%)]	Loss: 0.020242



Test set: Average loss: 0.0287, Accuracy: 9912/10000 (99%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.003134
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.001715
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.003132
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.013003
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.043803
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.000892
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.041307
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.000851
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.034990
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.000684
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.028237
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.002884
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.007012
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.004855
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.002370
Train Epoch: 8 [9600/60000 (16%)]	Loss: 0.001460
Train Epoch: 8 [10240/60000 (17%)]	Loss: 0.037944
Train Epoch: 8 [10880/60000 (18%)]	Loss: 0.011216
Train Epoch: 8 [11520/60000 (19%)]	Loss: 0.040602
Train Epoch: 8 [12

Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.102834
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.116895
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.021398
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.035847
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.003516
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.015847
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.004811
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.007094
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.066602
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.012162
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.001844
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.022196
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.056676
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.009672
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.010885
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.036050
Train Epoch: 9 [54400/60000 (91%)]	Loss: 0.007612
Train Epoch: 9 [55040/60000 (92%)]	Loss: 0.024670
Train Epoch: 9 [55680/60000 (93%)]	Loss: 0.033122
Train Epoch: 9 [56320/60000 (94%)]	Loss: 0.011193


Train Epoch: 11 [26880/60000 (45%)]	Loss: 0.020021
Train Epoch: 11 [27520/60000 (46%)]	Loss: 0.012968
Train Epoch: 11 [28160/60000 (47%)]	Loss: 0.003263
Train Epoch: 11 [28800/60000 (48%)]	Loss: 0.009870
Train Epoch: 11 [29440/60000 (49%)]	Loss: 0.070816
Train Epoch: 11 [30080/60000 (50%)]	Loss: 0.011272
Train Epoch: 11 [30720/60000 (51%)]	Loss: 0.015117
Train Epoch: 11 [31360/60000 (52%)]	Loss: 0.020433
Train Epoch: 11 [32000/60000 (53%)]	Loss: 0.063429
Train Epoch: 11 [32640/60000 (54%)]	Loss: 0.005651
Train Epoch: 11 [33280/60000 (55%)]	Loss: 0.053746
Train Epoch: 11 [33920/60000 (57%)]	Loss: 0.001302
Train Epoch: 11 [34560/60000 (58%)]	Loss: 0.004419
Train Epoch: 11 [35200/60000 (59%)]	Loss: 0.080869
Train Epoch: 11 [35840/60000 (60%)]	Loss: 0.007630
Train Epoch: 11 [36480/60000 (61%)]	Loss: 0.052876
Train Epoch: 11 [37120/60000 (62%)]	Loss: 0.097450
Train Epoch: 11 [37760/60000 (63%)]	Loss: 0.009090
Train Epoch: 11 [38400/60000 (64%)]	Loss: 0.027557
Train Epoch: 11 [39040/60000 (6

Train Epoch: 13 [9600/60000 (16%)]	Loss: 0.008836
Train Epoch: 13 [10240/60000 (17%)]	Loss: 0.146082
Train Epoch: 13 [10880/60000 (18%)]	Loss: 0.060885
Train Epoch: 13 [11520/60000 (19%)]	Loss: 0.017013
Train Epoch: 13 [12160/60000 (20%)]	Loss: 0.024372
Train Epoch: 13 [12800/60000 (21%)]	Loss: 0.008533
Train Epoch: 13 [13440/60000 (22%)]	Loss: 0.063231
Train Epoch: 13 [14080/60000 (23%)]	Loss: 0.001392
Train Epoch: 13 [14720/60000 (25%)]	Loss: 0.000657
Train Epoch: 13 [15360/60000 (26%)]	Loss: 0.006355
Train Epoch: 13 [16000/60000 (27%)]	Loss: 0.001468
Train Epoch: 13 [16640/60000 (28%)]	Loss: 0.052018
Train Epoch: 13 [17280/60000 (29%)]	Loss: 0.016931
Train Epoch: 13 [17920/60000 (30%)]	Loss: 0.013179
Train Epoch: 13 [18560/60000 (31%)]	Loss: 0.002698
Train Epoch: 13 [19200/60000 (32%)]	Loss: 0.096466
Train Epoch: 13 [19840/60000 (33%)]	Loss: 0.001715
Train Epoch: 13 [20480/60000 (34%)]	Loss: 0.008285
Train Epoch: 13 [21120/60000 (35%)]	Loss: 0.010413
Train Epoch: 13 [21760/60000 (36

Train Epoch: 14 [53760/60000 (90%)]	Loss: 0.043913
Train Epoch: 14 [54400/60000 (91%)]	Loss: 0.014814
Train Epoch: 14 [55040/60000 (92%)]	Loss: 0.000766
Train Epoch: 14 [55680/60000 (93%)]	Loss: 0.009339
Train Epoch: 14 [56320/60000 (94%)]	Loss: 0.003760
Train Epoch: 14 [56960/60000 (95%)]	Loss: 0.000918
Train Epoch: 14 [57600/60000 (96%)]	Loss: 0.001584
Train Epoch: 14 [58240/60000 (97%)]	Loss: 0.011969
Train Epoch: 14 [58880/60000 (98%)]	Loss: 0.002647
Train Epoch: 14 [59520/60000 (99%)]	Loss: 0.001473

Test set: Average loss: 0.0272, Accuracy: 9918/10000 (99%)



In [6]:
torch.cuda.current_device()

0

In [7]:
torch.cuda.device(0)

In [8]:
torch.cuda.device_count()

1

In [9]:
torch.cuda.get_device_name(0)

'GeForce RTX 2060'

In [10]:
torch.cuda.is_available()

True

In [5]:
train_kwargs = {'batch_size': 16}
test_kwargs = {'batch_size': 1000}
    
if True:
    cuda_kwargs = {'num_workers': 1,
                   'pin_memory': True,
                   'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                   transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                   transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

(60000, 28, 28)